In [1]:
#Importing dependencies
import numpy as np
import string
import random
import re
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.models import Sequential
from numpy import array, argmax, random, take
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RNN, SimpleRNN, LSTM,  Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
%matplotlib inline
#For plotting the matplotlib graphs in notebook

In [42]:
Data_path="C:\\Users\\SREEHARI\\Desktop\\internship\\my training\\Chapter12_RNN_LSTM_V4"

In [44]:
import pandas as pd
column_names = ['word1', 'word2', 'word3']

input_3gram = pd.read_csv(Data_path+ "\\Datasets\\3Gram_love_data.txt", delimiter='\t', names=column_names) #Importing csv file with column names
print("shape of data", input_3gram.shape)

shape of data (5351, 3)


In [4]:
print("Few sample records from data \n", input_3gram.sample(10))

Few sample records from data 
      word1 word2 word3
2360  love   him    so
1117  love    to   see
1257  love   him  with
160   hate    it  when
1791  love  them    so
199   hate  when  that
385   hate    to   say
1473  love  that     i
2553  love   the   way
2527  love   the   way


In [5]:
print("\nFrequency of word1 values \n", input_3gram["word1"].value_counts())
print("\nFrequency of word2 values \n", input_3gram["word2"].value_counts())


Frequency of word1 values 
 love      4327
loved      416
hate       400
hated       80
loves       72
lovely      24
loving      24
hates        8
Name: word1, dtype: int64

Frequency of word2 values 
 to         1866
it         1361
the         548
with        240
him         144
you         144
of          136
her         104
for          96
and          88
what         56
is           48
each         40
in           40
ones         32
me           32
nothing      32
them         32
as           24
every        24
more         16
going        16
that         16
being        16
affair       16
my           16
about         8
your          8
on            8
letter        8
most          8
thy           8
view          8
song          8
makes         8
got           8
this          8
at            8
a             8
one           8
hearing       8
story         8
all           8
lost          8
man           8
when          8
husband       8
Name: word2, dtype: int64


In [6]:
"""
Finding our words to create dictionary
Here we find unique values in each column and save each of those values .
Later which we will take the unique value for the entire appened columns
This will be our vocabulary list,which are the unique words in our data file
"""
unique_words = []
for i in list(input_3gram.columns.values):
    for j in pd.unique(input_3gram[i]):
        unique_words.append(j)
unique_words = np.unique(unique_words)


print('Count of unique words overall:', len(unique_words))
print('unique words list:', unique_words)

Count of unique words overall: 139
unique words list: ['a' 'able' 'about' 'admit' 'affair' 'affection' 'all' 'and' 'another'
 'answer' 'as' 'at' 'be' 'because' 'being' 'better' 'between' 'bother'
 'break' 'care' 'cared' 'come' 'concern' 'country' 'cut' 'disappoint' 'do'
 'each' 'every' 'fact' 'feel' 'feeling' 'find' 'first' 'for' 'from' 'get'
 'go' 'god' 'going' 'got' 'hate' 'hated' 'hates' 'have' 'he' 'hear'
 'hearing' 'her' 'here' 'him' 'his' 'husband' 'i' 'idea' 'if' 'in'
 'interrupt' 'is' 'it' 'kind' 'know' 'leave' 'letter' 'life' 'like'
 'listen' 'look' 'lost' 'lot' 'love' 'loved' 'lovely' 'loves' 'loving'
 'make' 'makes' 'man' 'marriage' 'me' 'minute' 'more' 'most' 'much'
 'music' 'my' 'nature' 'neighbor' 'not' 'nothing' 'of' 'on' 'one' 'ones'
 'or' 'other' 'over' 'play' 'respect' 'say' 'see' 'sit' 'smell' 'so'
 'someone' 'song' 'sound' 'story' 'stronger' 'support' 'take' 'talk'
 'tell' 'than' 'that' 'the' 'them' 'they' 'think' 'this' 'thought' 'thy'
 'to' 'too' 'united' 'use' 'v

In [7]:
"""
creating our word:indice pair dictionary and inverse
Here will be creating two dictonary values
word_indices : This contains each words mapped to an unique digit 
indices_words : This contains each digits mapped to a word in the same sequence as word_indices 
"""
word_indices = dict((w, i) for i, w in enumerate(unique_words))
indices_words = dict((i, w) for i, w in enumerate(unique_words))

print("word_indices dictionary \n",word_indices)
print("word_indices.keys \n", word_indices.keys())
print("word_indices.values \n", word_indices.values())
print("\n ########################################\n")
print("indices_words dictionary \n", indices_words)
print("indices_words keys \n",indices_words.keys())
print("indices_words values \n",indices_words.values())

word_indices dictionary 
 {'a': 0, 'able': 1, 'about': 2, 'admit': 3, 'affair': 4, 'affection': 5, 'all': 6, 'and': 7, 'another': 8, 'answer': 9, 'as': 10, 'at': 11, 'be': 12, 'because': 13, 'being': 14, 'better': 15, 'between': 16, 'bother': 17, 'break': 18, 'care': 19, 'cared': 20, 'come': 21, 'concern': 22, 'country': 23, 'cut': 24, 'disappoint': 25, 'do': 26, 'each': 27, 'every': 28, 'fact': 29, 'feel': 30, 'feeling': 31, 'find': 32, 'first': 33, 'for': 34, 'from': 35, 'get': 36, 'go': 37, 'god': 38, 'going': 39, 'got': 40, 'hate': 41, 'hated': 42, 'hates': 43, 'have': 44, 'he': 45, 'hear': 46, 'hearing': 47, 'her': 48, 'here': 49, 'him': 50, 'his': 51, 'husband': 52, 'i': 53, 'idea': 54, 'if': 55, 'in': 56, 'interrupt': 57, 'is': 58, 'it': 59, 'kind': 60, 'know': 61, 'leave': 62, 'letter': 63, 'life': 64, 'like': 65, 'listen': 66, 'look': 67, 'lost': 68, 'lot': 69, 'love': 70, 'loved': 71, 'lovely': 72, 'loves': 73, 'loving': 74, 'make': 75, 'makes': 76, 'man': 77, 'marriage': 78,

In [8]:
### Onehot encoding of word1
word1 = input_3gram['word1'].map(word_indices)
word1_onehot = keras.utils.to_categorical(np.array(word1), num_classes=len(word_indices))
print("word1_onehot shape is ",word1_onehot.shape)

word1_onehot shape is  (5351, 139)


In [9]:
#Lets take example of two different words
print("The word in row 0 is -->"+input_3gram['word1'][0])
print("The one hot encoded version of the word in row 0 is \n",word1_onehot[0])

print("\nThe word in row 500 is --> "+input_3gram['word1'][500])
print("The one hot encoded version of the word in row 500 is \n",word1_onehot[500])

The word in row 0 is -->hate
The one hot encoded version of the word in row 0 is 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

The word in row 500 is --> love
The one hot encoded version of the word in row 500 is 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [10]:
##one hot encoding for word2 and word3 
word2 = input_3gram['word2'].map(word_indices)
word2_onehot = keras.utils.to_categorical(np.array(word2), num_classes=len(word_indices))
print("word2_onehot shape is ",word2_onehot.shape)

word3 = input_3gram['word3'].map(word_indices)
word3_onehot = keras.utils.to_categorical(np.array(word3), num_classes=len(word_indices))
print("word3_onehot shape is ",word3_onehot.shape)

word2_onehot shape is  (5351, 139)
word3_onehot shape is  (5351, 139)


### First ANN Model

In [11]:
ANN_model1 = Sequential()
ANN_model1.add(Dense(10, input_dim=word1_onehot.shape[1], activation='sigmoid'))
ANN_model1.add(Dense(word2_onehot.shape[1] ,activation='softmax'))
ANN_model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1400      
_________________________________________________________________
dense_1 (Dense)              (None, 139)               1529      
Total params: 2,929
Trainable params: 2,929
Non-trainable params: 0
_________________________________________________________________


In [12]:
ANN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model1.fit(word1_onehot, word2_onehot, epochs=20, batch_size=50,  verbose=1)

Epoch 1/20
108/108 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.0015
Epoch 2/20
108/108 [==============================] - 0s 2ms/step - loss: 0.3207 - accuracy: 0.0639
Epoch 3/20
108/108 [==============================] - 0s 2ms/step - loss: 0.1870 - accuracy: 0.3487
Epoch 4/20
108/108 [==============================] - 0s 2ms/step - loss: 0.1194 - accuracy: 0.3487
Epoch 5/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0844 - accuracy: 0.3487
Epoch 6/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0649 - accuracy: 0.3487
Epoch 7/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0531 - accuracy: 0.3487
Epoch 8/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0455 - accuracy: 0.3487
Epoch 9/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0404 - accuracy: 0.3487
Epoch 10/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0368 - accuracy: 0.3487

In [13]:
#We will see what the 1st hidden layer output representation of the data  
# to predict the hidden layer activations, 
# let's rewrite first layer of our model and give it the weights from fully trained previous model
model1_hidden = Sequential()
model1_hidden.add(Dense(10, input_dim=word1_onehot.shape[1], weights=ANN_model1.layers[0].get_weights()))
model1_hidden.add(Activation('sigmoid'))

In [14]:
# Getting the hidden layer activations
model1_hidden_output = model1_hidden.predict(word1_onehot)
#peak into our hidden layer activations
print("The hidden layer output for every record - Shape of it \n", model1_hidden_output.shape)
print("Few five records from hidden layer \n",model1_hidden_output[:5])

The hidden layer output for every record - Shape of it 
 (5351, 10)
Few five records from hidden layer 
 [[0.8334368  0.84318197 0.8652694  0.8443494  0.8454185  0.8570777
  0.84676427 0.8684742  0.8556158  0.8603386 ]
 [0.8334368  0.84318197 0.8652694  0.8443494  0.8454185  0.8570777
  0.84676427 0.8684742  0.8556158  0.8603386 ]
 [0.8334368  0.84318197 0.8652694  0.8443494  0.8454185  0.8570777
  0.84676427 0.8684742  0.8556158  0.8603386 ]
 [0.8334368  0.84318197 0.8652694  0.8443494  0.8454185  0.8570777
  0.84676427 0.8684742  0.8556158  0.8603386 ]
 [0.8334368  0.84318197 0.8652694  0.8443494  0.8454185  0.8570777
  0.84676427 0.8684742  0.8556158  0.8603386 ]]


In [15]:
"""
We append the input words of the words2 column in the output of the h1 layer,this gives us the combined input representation
"""
word2_hidden_append = np.append(model1_hidden_output,word2_onehot, axis=1)
print("word2_hidden_append Shape", word2_hidden_append.shape)

word2_hidden_append Shape (5351, 149)


In [16]:
ANN_model2 = Sequential()
ANN_model2.add(Dense(10, input_dim=word2_hidden_append.shape[1], activation='sigmoid'))
ANN_model2.add(Dense(word3_onehot.shape[1], activation='softmax'))
ANN_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                1500      
_________________________________________________________________
dense_4 (Dense)              (None, 139)               1529      
Total params: 3,029
Trainable params: 3,029
Non-trainable params: 0
_________________________________________________________________


In [17]:
ANN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = ANN_model2.fit(word2_hidden_append, word3_onehot, epochs=20, batch_size=50,  verbose=1)

Epoch 1/20
108/108 [==============================] - 0s 2ms/step - loss: 0.5060 - accuracy: 0.0015
Epoch 2/20
108/108 [==============================] - 0s 2ms/step - loss: 0.2260 - accuracy: 0.0890
Epoch 3/20
108/108 [==============================] - 0s 2ms/step - loss: 0.1201 - accuracy: 0.2439
Epoch 4/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0811 - accuracy: 0.2439
Epoch 5/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0627 - accuracy: 0.2439
Epoch 6/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0526 - accuracy: 0.2439
Epoch 7/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0464 - accuracy: 0.2439
Epoch 8/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0424 - accuracy: 0.2439
Epoch 9/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0396 - accuracy: 0.2439
Epoch 10/20
108/108 [==============================] - 0s 2ms/step - loss: 0.0376 - accuracy: 0.2439

In [18]:
# A predict function that takes input word1 and word2; and predict word3 
#1. take the input word , and represent them using digits from the word_indices dictonary values
#2. getting the intermediate hidden nodes for word1
#3. appending hidden activations with word2 as final test set
#4. prediction on this test set
def two_step_pred(words_in):

    index_input=word_indices[words_in[0]]
    indices_in = keras.utils.to_categorical(index_input, num_classes=len(word_indices))
    indices_in=indices_in.reshape(1,len(word_indices))
    h1_test = model1_hidden.predict(indices_in) # getting our intermediate hidden activations from model1h
    
    
    index_input2=word_indices[words_in[1]]
    indices_in2 = keras.utils.to_categorical(index_input2, num_classes=len(word_indices))
    indices_in2= indices_in2.reshape(1,len(word_indices))
    X2_test = np.append(h1_test, indices_in2, axis=1) #preparing final test data by appending hidden with word2
    
    yhat = ANN_model2.predict_classes(X2_test) #predicting final output from model2
    
    print("Input words --> ", words_in)
    print("Predicted word --> ", indices_words[yhat[0]])

In [19]:
two_step_pred(['love', 'it'])
two_step_pred(['love', 'to'])
two_step_pred(['love', 'the'])

Input words -->  ['love', 'it']
Predicted word -->  when
Input words -->  ['love', 'to']
Predicted word -->  when
Input words -->  ['love', 'the']
Predicted word -->  when


C:\Users\SREEHARI\anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## RNNs

In [20]:
model = Sequential()
model.add(SimpleRNN(4, use_bias=False, input_shape=(2,2)))
model.add(Dense(3, use_bias=False, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 24        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 12        
Total params: 36
Trainable params: 36
Non-trainable params: 0
_________________________________________________________________


In [21]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(2,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 15        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [22]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(4,2)))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 4)                 28        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 15        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


## Word prediction using RNN model

In [23]:
word1_word2 = input_3gram[['word1','word2']]
for i in list(word1_word2.columns.values):
    word1_word2[i] = word1_word2[i].map(word_indices)

word1_word2=np.array(word1_word2)
#The same data is reshaped with similar structure but appended with 1 value to make it 3d array
word1_word2=np.reshape(word1_word2,(word1_word2.shape[0],2,1))
word1_word2_onehot = keras.utils.to_categorical(np.array(word1_word2), num_classes=len(word_indices))
print("word1_word2_onehot shape", word1_word2_onehot.shape)

word1_word2_onehot shape (5351, 2, 139)


C:\Users\SREEHARI\AppData\Local\Temp\ipykernel_24376\3906608908.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word1_word2[i] = word1_word2[i].map(word_indices)
C:\Users\SREEHARI\AppData\Local\Temp\ipykernel_24376\3906608908.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word1_word2[i] = word1_word2[i].map(word_indices)


In [24]:
print("time steps" , word1_word2_onehot.shape[1])
print("Input nodes" , word1_word2_onehot.shape[2])
print("output nodes" , word3_onehot.shape[1])

time steps 2
Input nodes 139
output nodes 139


In [25]:
model_rnn = Sequential()
#model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, input_data_dim)))
model_rnn.add(SimpleRNN(30, input_shape=(word1_word2_onehot.shape[1],word1_word2_onehot.shape[2]))) 
model_rnn.add(Dense(word3_onehot.shape[1], activation='softmax'))
model_rnn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 30)                5100      
_________________________________________________________________
dense_8 (Dense)              (None, 139)               4309      
Total params: 9,409
Trainable params: 9,409
Non-trainable params: 0
_________________________________________________________________


In [26]:
# compile network
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_rnn.fit(word1_word2_onehot, word3_onehot, epochs=20)

Epoch 1/20
168/168 [==============================] - 1s 3ms/step - loss: 3.7604 - accuracy: 0.2801
Epoch 2/20
168/168 [==============================] - 1s 3ms/step - loss: 2.8047 - accuracy: 0.4743
Epoch 3/20
168/168 [==============================] - 1s 3ms/step - loss: 2.3615 - accuracy: 0.5317
Epoch 4/20
168/168 [==============================] - 1s 3ms/step - loss: 2.0610 - accuracy: 0.5689
Epoch 5/20
168/168 [==============================] - 1s 3ms/step - loss: 1.8732 - accuracy: 0.5905
Epoch 6/20
168/168 [==============================] - 1s 3ms/step - loss: 1.7375 - accuracy: 0.6066
Epoch 7/20
168/168 [==============================] - 0s 3ms/step - loss: 1.6322 - accuracy: 0.6225
Epoch 8/20
168/168 [==============================] - 0s 3ms/step - loss: 1.5455 - accuracy: 0.6361
Epoch 9/20
168/168 [==============================] - 0s 3ms/step - loss: 1.4739 - accuracy: 0.6481
Epoch 10/20
168/168 [==============================] - 0s 3ms/step - loss: 1.4153 - accuracy: 0.6567

In [27]:
def rnn_word_pred(in_text):
    print("Input is - " , in_text)
    encoded = [word_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded =keras.utils.to_categorical(np.array(encoded), num_classes=len(word_indices))
    ypred = model_rnn.predict_classes(encoded, verbose=0)[0]
    print("Output is --> " ,indices_words[ypred])

In [28]:
rnn_word_pred(['love', 'it'])
rnn_word_pred(['love', 'to'])
rnn_word_pred(['love', 'the'])

Input is -  ['love', 'it']
Output is -->  when
Input is -  ['love', 'to']
Output is -->  see
Input is -  ['love', 'the']
Output is -->  way


C:\Users\SREEHARI\anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## RNN for Long Sequences

In [29]:
longseq_3gram = open(Data_path+'\\Long_sequence_3gram.csv').read().lower()
print(longseq_3gram[495:801])
print(longseq_3gram[30615:31000])


a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of
a,combination,of

and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,according,to
and,addresses,of
and,adherence,to
and,advocates,for
and,aerospace,engineering
and,americans,do
and,analyzing,the
and,announced,he
and,announced,he
and,announced,plans
and,announced,that
and,announced,that
and,annou


In [30]:
#Replace comma with space
longseq_3gram1= longseq_3gram.replace(',',' ').replace('\r','')
print(longseq_3gram1[495:750])
print(longseq_3gram1[30615:30800])


a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of
a combination of

and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and according to
and addresses 


In [31]:
#Unique characters in our dataset we then sort it
chars = sorted(list(set(longseq_3gram1)))
print("Unique Characters in the text \n ",chars)
#\n is character string for new line, we dont need that in our dictionary of chars
chars.remove('\n')
print("\n Character after removing newline symbol \'\\n\'",chars)
print("\n overall chars count", len(chars))

Unique Characters in the text 
  ['\n', ' ', "'", '(', '-', '.', '/', '0', '1', '3', '7', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 Character after removing newline symbol '\n' [' ', "'", '(', '-', '.', '/', '0', '1', '3', '7', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 overall chars count 37


In [32]:
char_indices = dict((c, i) for i, c in enumerate(chars))
print("characters to indices dictionary\n", char_indices)
indices_char = dict((i, c) for i, c in enumerate(chars))
print("indices to char dictionary\n", indices_char)
print('unique chars: ', {len(chars)})

characters to indices dictionary
 {' ': 0, "'": 1, '(': 2, '-': 3, '.': 4, '/': 5, '0': 6, '1': 7, '3': 8, '7': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36}
indices to char dictionary
 {0: ' ', 1: "'", 2: '(', 3: '-', 4: '.', 5: '/', 6: '0', 7: '1', 8: '3', 9: '7', 10: '9', 11: 'a', 12: 'b', 13: 'c', 14: 'd', 15: 'e', 16: 'f', 17: 'g', 18: 'h', 19: 'i', 20: 'j', 21: 'k', 22: 'l', 23: 'm', 24: 'n', 25: 'o', 26: 'p', 27: 'q', 28: 'r', 29: 's', 30: 't', 31: 'u', 32: 'v', 33: 'w', 34: 'x', 35: 'y', 36: 'z'}
unique chars:  {37}


### Char to index on full data

In [33]:
data = longseq_3gram1.splitlines()
##Adding a space at the end
data = [i+' ' for i in data]

##mapping our data into numbers
sentences = [[char_indices[j] for j in i] for i in data ]
print(data[0], sentences[0])
print(data[10], sentences[1])
print(data[20], sentences[2])
print(data[100], sentences[3])
print(data[400], sentences[400])
print(data[4000], sentences[4000])
print(data[9000], sentences[9000])
##Number of sentences
print("Number of sentences ", len(sentences))

a bewildering array  [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35, 0]
a celebration of  [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16, 0]
a co-director of  [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 32, 11, 28, 19, 15, 30, 35, 0]
a declaration of  [11, 0, 12, 19, 30, 30, 15, 28, 29, 33, 15, 15, 30, 0, 23, 25, 23, 15, 24, 30, 0]
a significant risk  [11, 0, 29, 19, 17, 24, 19, 16, 19, 13, 11, 24, 30, 0, 28, 19, 29, 21, 0]
been designed as  [12, 15, 15, 24, 0, 14, 15, 29, 19, 17, 24, 15, 14, 0, 11, 29, 0]
from anywhere on  [16, 28, 25, 23, 0, 11, 24, 35, 33, 18, 15, 28, 15, 0, 25, 24, 0]
Number of sentences  30307


### Converting the sentences to RNN friendly data which can be used to generate new sequences

**Take one sentence, iterate through it till the length of sentence is reached:**

* **Step 1** 0:0+14: X; and 14th position: y >> observation 1
* **Step 2** 1:14: X; and 15th position: y >> observation 2
* **Step 3** …we do this till the length of sentence is reached

Take next sentence and repeat the same


In [34]:
#Since all the sentences may not be of same length,it is neccessary to make them consistent when passing to keras
#We select a sequence length
Seq_ln = 14
X = []
y = []
for i in sentences:
    for j in range(len(i)-Seq_ln):
        X.append(i[j:j+Seq_ln])
        y.append(i[j+Seq_ln])
len(X), len(y)

(142142, 142142)

### Sample

In [35]:
print("data[0:2]=", data[0:2])
print("sentences[0:2]=", sentences[0:2])

for i in range (0,20):
    print("X[",i,"]=", X[i],"y[",i,"]=", y[i])

data[0:2]= ['a bewildering array ', 'a beneficiary of ']
sentences[0:2]= [[11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35, 0], [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16, 0]]
X[ 0 ]= [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0] y[ 0 ]= 11
X[ 1 ]= [0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11] y[ 1 ]= 28
X[ 2 ]= [12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28] y[ 2 ]= 28
X[ 3 ]= [15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28] y[ 3 ]= 11
X[ 4 ]= [33, 19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11] y[ 4 ]= 35
X[ 5 ]= [19, 22, 14, 15, 28, 19, 24, 17, 0, 11, 28, 28, 11, 35] y[ 5 ]= 0
X[ 6 ]= [11, 0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0] y[ 6 ]= 25
X[ 7 ]= [0, 12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25] y[ 7 ]= 16
X[ 8 ]= [12, 15, 24, 15, 16, 19, 13, 19, 11, 28, 35, 0, 25, 16] y[ 8 ]= 0
X[ 9 ]= [11, 0, 12, 15, 33, 19, 22, 14, 15, 28, 19, 24, 17, 0] y[ 9 ]= 32
X[ 10 ]= [0, 12, 15, 33, 19, 22, 14, 

In [36]:
#The first row is the X's first row up to 14 character
#The second row is the X's first row starting from second character up to 14 character
#The third row is the X's first row starting from third character up to 14 character and so on 
X=np.array(X)
X1=np.reshape(X,(X.shape[0],X.shape[1],1))
X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
print(X1.shape)

(142142, 14, 37)


In [37]:
#Target Variable
y[:10]
#Reshapig our label for model
y1 = np.array(y)
# one hot encode outputs
y1 = keras.utils.to_categorical(np.array(y), num_classes=len(char_indices))
y1.shape

(142142, 37)

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.20)
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

X_train.shape (113713, 14, 37)
y_train.shape (113713, 37)
X_test.shape (28429, 14, 37)
y_test.shape (28429, 37)


In [39]:
#building the model
model_RNN2 = Sequential()
##model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_RNN2.add(SimpleRNN(16, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_RNN2.add(Dense(len(char_indices)))
model_RNN2.add(Activation('softmax'))
model_RNN2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 16)                864       
_________________________________________________________________
dense_9 (Dense)              (None, 37)                629       
_________________________________________________________________
activation_1 (Activation)    (None, 37)                0         
Total params: 1,493
Trainable params: 1,493
Non-trainable params: 0
_________________________________________________________________


In [40]:
# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_test, y_test))
model_RNN2.save_weights("char_rnn_model_weights_v1.hdf5")

Epoch 1/30
3554/3554 [==============================] - 38s 10ms/step - loss: 2.2602 - accuracy: 0.3648 - val_loss: 1.9787 - val_accuracy: 0.4235
Epoch 2/30
3554/3554 [==============================] - 37s 10ms/step - loss: 1.9122 - accuracy: 0.4432 - val_loss: 1.8757 - val_accuracy: 0.4456
Epoch 3/30
3554/3554 [==============================] - 37s 10ms/step - loss: 1.8392 - accuracy: 0.4596 - val_loss: 1.8249 - val_accuracy: 0.4625
Epoch 4/30
3554/3554 [==============================] - 37s 10ms/step - loss: 1.7992 - accuracy: 0.4714 - val_loss: 1.7914 - val_accuracy: 0.4702
Epoch 5/30
3554/3554 [==============================] - 37s 10ms/step - loss: 1.7723 - accuracy: 0.4792 - val_loss: 1.7694 - val_accuracy: 0.4788
Epoch 6/30
3554/3554 [==============================] - 38s 11ms/step - loss: 1.7526 - accuracy: 0.4836 - val_loss: 1.7520 - val_accuracy: 0.4848
Epoch 7/30
3554/3554 [==============================] - 36s 10ms/step - loss: 1.7380 - accuracy: 0.4868 - val_loss: 1.7394 -

In [45]:
weightsfile_model_RNN2= Data_path+"\\Pre_trained_models\\char_rnn_model_weights_v1.hdf5"
model_RNN2.load_weights(weightsfile_model_RNN2)

# compile network
model_RNN2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_RNN2.fit(X_train, y_train, epochs=2, verbose=1)

Epoch 1/2
3554/3554 [==============================] - 37s 10ms/step - loss: 1.6380 - accuracy: 0.5120
Epoch 2/2
3554/3554 [==============================] - 38s 11ms/step - loss: 1.6361 - accuracy: 0.5125


In [46]:
#function to prepare test input
def prepare_input(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input(in_text)
        pred = model_RNN2.predict_classes(x, verbose=0)[0]

        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [47]:
in_text = 'officials say '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted word ---> ", out_word)
in_text = 'how dangerous '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'political and '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'whatever they '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)
in_text = 'of particular '
out_word = complete_pred(in_text)
print("Input text -->", in_text, "\npredicted output ---> ", out_word)

C:\Users\SREEHARI\anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Input text --> officials say  
predicted word --->  to 
Input text --> how dangerous  
predicted output --->  of 
Input text --> political and  
predicted output --->  the 
Input text --> whatever they  
predicted output --->  the 
Input text --> of particular  
predicted output --->  to 


## LSTM Models

In [48]:
#building the model
model_LSTM = Sequential()
#model1.add(LSTM('number of hidden nodes in each rnn cell', input_shape=(timesteps, data_dim)))
model_LSTM.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_LSTM.add(Dense(len(char_indices)))
model_LSTM.add(Activation('softmax'))
model_LSTM.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               84992     
_________________________________________________________________
dense_10 (Dense)             (None, 37)                4773      
_________________________________________________________________
activation_2 (Activation)    (None, 37)                0         
Total params: 89,765
Trainable params: 89,765
Non-trainable params: 0
_________________________________________________________________


In [49]:
# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train, epochs=30, verbose=1)
model_LSTM.save_weights("char_LSTM_model_weights_v1.hdf5")

Epoch 1/30
3554/3554 [==============================] - 17s 4ms/step - loss: 1.9520 - accuracy: 0.4304
Epoch 2/30
3554/3554 [==============================] - 16s 4ms/step - loss: 1.5527 - accuracy: 0.5372
Epoch 3/30
3554/3554 [==============================] - 16s 5ms/step - loss: 1.3849 - accuracy: 0.5876
Epoch 4/30
3554/3554 [==============================] - 17s 5ms/step - loss: 1.2700 - accuracy: 0.6200
Epoch 5/30
3554/3554 [==============================] - 17s 5ms/step - loss: 1.1837 - accuracy: 0.6454
Epoch 6/30
3554/3554 [==============================] - 16s 4ms/step - loss: 1.1157 - accuracy: 0.6660
Epoch 7/30
3554/3554 [==============================] - 15s 4ms/step - loss: 1.0600 - accuracy: 0.6827
Epoch 8/30
3554/3554 [==============================] - 16s 5ms/step - loss: 1.0126 - accuracy: 0.6963
Epoch 9/30
3554/3554 [==============================] - 17s 5ms/step - loss: 0.9708 - accuracy: 0.7077
Epoch 10/30
3554/3554 [==============================] - 15s 4ms/step - l

In [50]:
weightsfile_model_LSTM= Data_path+"\\Pre_trained_models\\char_LSTM_model_weights_v1.hdf5"
model_LSTM.load_weights( weightsfile_model_LSTM)

# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train,epochs=2, verbose=1)

Epoch 1/2
3554/3554 [==============================] - 17s 4ms/step - loss: 0.7146 - accuracy: 0.7902
Epoch 2/2
3554/3554 [==============================] - 17s 5ms/step - loss: 0.6708 - accuracy: 0.7954


In [51]:
#function to prepare test input
def prepare_input1(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1= keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred1(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input1(in_text)
        pred = model_LSTM.predict_classes(x, verbose=0)[0]
        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [52]:
in_text = 'the emergence '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'officials say '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and sentenced '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'a combination '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'and according '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)

Input text --> the emergence  ; predicted output --->  of 
Input text --> officials say  ; predicted output --->  that 
Input text --> and sentenced  ; predicted output --->  to 
Input text --> a combination  ; predicted output --->  of 
Input text --> and according  ; predicted output --->  to 


### Few more predictions and Comparions with Standard RNN

In [53]:
in_text = 'how dangerous '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'political and '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'of particular '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

print("\n")
in_text = 'whatever they '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "\nLSTM Prediction ---> ", out_word)
out_word1 = complete_pred(in_text)
print("RNN Prediction ---> ", out_word1)

Input text --> how dangerous  
LSTM Prediction --->  is 
RNN Prediction --->  of 


Input text --> political and  
LSTM Prediction --->  economic 
RNN Prediction --->  the 


Input text --> of particular  
LSTM Prediction --->  importance 
RNN Prediction --->  to 


Input text --> whatever they  
LSTM Prediction --->  want 
RNN Prediction --->  the 


## Case Study – Language Translation Project 

In [79]:
raw_data= open(Data_path+"\\Datasets\\fra-eng\\eng-fra.txt", mode='rt', encoding='utf-8').read()
raw_data=raw_data.strip().split('\n')
raw_data=[i.split('\t') for i in raw_data]
lang1_lang2_data=array(raw_data)
print(lang1_lang2_data)
print("Overall pairs", len(lang1_lang2_data))

[['Go.' 'Va !']
 ['Run!' 'Cours\u202f!']
 ['Run!' 'Courez\u202f!']
 ...
 ['Since there are usually multiple websites on any given topic, I usually just click the back button when I arrive on any webpage that has pop-up advertising. I just go to the next page found by Google and hope for something less irritating.'
  "Puisqu'il y a de multiples sites web sur chaque sujet, je clique d'habitude sur le bouton retour arrière lorsque j'atterris sur n'importe quelle page qui contient des publicités surgissantes. Je me rends juste sur la prochaine page proposée par Google et espère tomber sur quelque chose de moins irritant."]
 ["If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker."
  "Si quelqu'un qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif, cela veut 

In [80]:
# Remove punctuation
lang1_lang2_data[:,0] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,0]]
lang1_lang2_data[:,1] = [word.translate(str.maketrans('', '', string.punctuation)) for word in lang1_lang2_data[:,1]]

print(lang1_lang2_data)

[['Go' 'Va ']
 ['Run' 'Cours\u202f']
 ['Run' 'Courez\u202f']
 ...
 ['Since there are usually multiple websites on any given topic I usually just click the back button when I arrive on any webpage that has popup advertising I just go to the next page found by Google and hope for something less irritating'
  'Puisquil y a de multiples sites web sur chaque sujet je clique dhabitude sur le bouton retour arrière lorsque jatterris sur nimporte quelle page qui contient des publicités surgissantes Je me rends juste sur la prochaine page proposée par Google et espère tomber sur quelque chose de moins irritant']
 ['If someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker In other words you dont really sound like a native speaker'
  'Si quelquun qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif cela veut dire quil a probablement r

In [81]:
## convert text to lowercase
for word in range(len(lang1_lang2_data)):
    lang1_lang2_data[word,0] = lang1_lang2_data[word,0].lower()
    lang1_lang2_data[word,1] = lang1_lang2_data[word,1].lower()
print(lang1_lang2_data)

[['go' 'va ']
 ['run' 'cours\u202f']
 ['run' 'courez\u202f']
 ...
 ['since there are usually multiple websites on any given topic i usually just click the back button when i arrive on any webpage that has popup advertising i just go to the next page found by google and hope for something less irritating'
  'puisquil y a de multiples sites web sur chaque sujet je clique dhabitude sur le bouton retour arrière lorsque jatterris sur nimporte quelle page qui contient des publicités surgissantes je me rends juste sur la prochaine page proposée par google et espère tomber sur quelque chose de moins irritant']
 ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker'
  'si quelquun qui ne connaît pas vos antécédents dit que vous parlez comme un locuteur natif cela veut dire quil a probablement r

In [82]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 0])
lang1_tokens=tokenizer
lang1_vocab_size = len(lang1_tokens.word_index) + 1
print("lang1_vocab_size", lang1_vocab_size)

lang1_vocab_size 13711


In [83]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lang1_lang2_data[:, 1])
lang2_tokens=tokenizer
lang2_vocab_size = len(lang2_tokens.word_index) + 1
print("lang2_vocab_size", lang2_vocab_size)

lang2_vocab_size 29664


In [84]:
# split data into train and test set
train, test = train_test_split(lang1_lang2_data, test_size=0.1, random_state = 44)

In [85]:
lang1_seq_length=15
lang2_seq_length=15

X_train_seq=lang1_tokens.texts_to_sequences(train[:, 0])
X_train= pad_sequences(X_train_seq,lang1_seq_length,padding='post')

Y_train_seq=lang2_tokens.texts_to_sequences(train[:, 1])
Y_train= pad_sequences(Y_train_seq,lang2_seq_length,padding='post')

X_test_seq=lang1_tokens.texts_to_sequences(test[:, 0])
X_test= pad_sequences(X_test_seq,lang1_seq_length,padding='post')

Y_test_seq=lang2_tokens.texts_to_sequences(test[:, 1])
Y_test= pad_sequences(Y_test_seq,lang2_seq_length,padding='post')

print("X_train.shape", X_train.shape)
print("Y_train.shape",Y_train.shape)
print("X_test.shape",X_test.shape)
print("Y_test.shape", Y_test.shape)

X_train.shape (122257, 15)
Y_train.shape (122257, 15)
X_test.shape (13585, 15)
Y_test.shape (13585, 15)


In [86]:
print("Text data", [train[5, 0]])
print('Numbers sequence', X_train_seq[5])
print('Padded Sequence', X_train[5])

Text data ['this song is familiar to us']
Numbers sequence [17, 615, 6, 1156, 3, 87]
Padded Sequence [  17  615    6 1156    3   87    0    0    0    0    0    0    0    0
    0]


### Model Building

In [87]:
model = Sequential()
model.add(Embedding(lang1_vocab_size, 256, input_length=lang1_seq_length, mask_zero=True))
model.add(LSTM(128))
model.add(RepeatVector(lang2_seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(lang2_vocab_size, activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 256)           3510016   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               197120    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 15, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 15, 128)           131584    
_________________________________________________________________
dense_13 (Dense)             (None, 15, 29664)         3826656   
Total params: 7,665,376
Trainable params: 7,665,376
Non-trainable params: 0
_________________________________________________________________


In [88]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
history = model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1),  epochs=1, verbose=1, batch_size=1024)
model.save_weights('Eng_fra_model.hdf5')

120/120 [==============================] - 21s 141ms/step - loss: 5.0375


In [95]:
model.load_weights("C:\\Users\\SREEHARI\\Desktop\\internship\\my training\\Chapter12_RNN_LSTM_V4\\Eng_fra_model.hdf5")

### Prediction

In [96]:
def one_line_prediction(text1):
    
    def to_lines(text):
          sents = text.strip().split('\n')
          sents = [i.split('\t') for i in sents]
          return sents
    small_input = to_lines(text1)
    small_input = array(small_input)
    
    # Remove punctuation
    small_input[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in small_input[:,0]]
    # convert text to lowercase
    for i in range(len(small_input)):
        small_input[i,0] = small_input[i,0].lower()

    #encode and pad sequences
    small_input_seq=lang1_tokens.texts_to_sequences(small_input[0])
    small_input= pad_sequences(small_input_seq,lang1_seq_length,padding='post')
   

    #Load the model
    #Eng French Model
    #model.load_weights('C:\\Users\\SREEHARI\\Desktop\\internship\\my training\\Chapter12_RNN_LSTM_V4\\Eng_fra_model.hdf5')

    pred_seq = model.predict_classes(small_input[0:1].reshape((small_input[0:1].shape[0],small_input[0:1].shape[1])))
    
    def num_to_word(n, tokens):
          for word, index in tokens.word_index.items():
              if index == n:
                  return word
          return None

    Lang2_text = []
    for word_num in pred_seq:
          sing_pred = []
          for i in range(len(word_num)):
                t = num_to_word(word_num[i], lang2_tokens)
                if i > 0:
                    if (t == num_to_word(word_num[i-1], lang2_tokens)) or (t == None):
                        sing_pred.append('')
                    else:
                        sing_pred.append(t)
                else:
                      if(t == None):
                              sing_pred.append('')
                      else:
                              sing_pred.append(t) 
          Lang2_text.append(' '.join(sing_pred))
    return(Lang2_text)

In [93]:
Input_sentences=["have a great Good day",
                 "Do you speak English",
                 "I do not know your language",
                 "I need help",
                 "Thank you very much",
                 "Where can I get this",
                 "How much does it cost",
                 "Where is the bathroom",
                 "Where is the ATM",
                 "I am a visitor here",
                 "Excuse me",
                 "What do you do for living",
                 "Here is my passport"]

for sent in Input_sentences:
  print([sent] , " -->",one_line_prediction(sent))

['have a great Good day']  --> ['              ']
['Do you speak English']  --> ['              ']
['I do not know your language']  --> ['              ']
['I need help']  --> ['              ']
['Thank you very much']  --> ['              ']
['Where can I get this']  --> ['              ']
['How much does it cost']  --> ['              ']
['Where is the bathroom']  --> ['              ']
['Where is the ATM']  --> ['              ']
['I am a visitor here']  --> ['              ']
['Excuse me']  --> ['              ']
['What do you do for living']  --> ['              ']
['Here is my passport']  --> ['              ']
